<a href="https://colab.research.google.com/github/AnkitaSK/Proj3_RoboReviews/blob/main/ReviewsSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl
from transformers import pipeline

In [ ]:
print(pl.__version__)

1.9.0


In [ ]:
!pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 59.4 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.9.0
    Uninstalling polars-1.9.0:
      Successfully uninstalled polars-1.9.0


In [ ]:
# Load large CSV file
df = pl.read_csv("reviews_with_Positive_topics.csv")

In [ ]:
df_filtered = df.filter(pl.col("Topic") != -1)

In [ ]:
print(df_filtered.head)

<bound method DataFrame.head of shape: (13_687, 27)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
│ id         ┆ dateAdded  ┆ dateUpdate ┆ name      ┆ … ┆ sourceURL ┆ reviews.r ┆ Sentiment ┆ Topic │
│ ---        ┆ ---        ┆ d          ┆ ---       ┆   ┆ s         ┆ ating_upd ┆ ---       ┆ ---   │
│ str        ┆ str        ┆ ---        ┆ str       ┆   ┆ ---       ┆ ate       ┆ str       ┆ i64   │
│            ┆            ┆ str        ┆           ┆   ┆ str       ┆ ---       ┆           ┆       │
│            ┆            ┆            ┆           ┆   ┆           ┆ i64       ┆           ┆       │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
│ AVpgNzjwLJ ┆ 2015-10-30 ┆ 2019-04-25 ┆ AmazonBas ┆ … ┆ https://w ┆ 2         ┆ Positive  ┆ 5     │
│ eJML43Kpxn ┆ T08:59:32Z ┆ T09:08:16Z ┆ ics AAA   ┆   ┆ ww.barcod ┆           ┆           ┆       │
│            ┆            ┆            

In [ ]:
print(type(df_filtered))

<class 'polars.dataframe.frame.DataFrame'>


In [ ]:
print(df_filtered.columns)

['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs', 'reviews.rating_update', 'Sentiment', 'Topic']


In [ ]:
# count the occurence of each topic
topic_counts = df_filtered.group_by("Topic").agg(pl.col("Topic").count().alias("count"))

# Sort by the count in descending order
topic_counts = topic_counts.sort("count", descending=True)
print(topic_counts)

shape: (46, 2)
┌───────┬───────┐
│ Topic ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3774  │
│ 1     ┆ 1172  │
│ 2     ┆ 648   │
│ 3     ┆ 624   │
│ 4     ┆ 554   │
│ …     ┆ …     │
│ 41    ┆ 90    │
│ 42    ┆ 85    │
│ 43    ┆ 83    │
│ 44    ┆ 83    │
│ 45    ┆ 80    │
└───────┴───────┘


In [ ]:
# Step 2: Select the top_n topics
top_n = 8
top_topics = topic_counts.head(top_n)["Topic"]

#### using facebook/bart-large-cnn

In [ ]:
# Step 3: Summarize reviews for each topic
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = {}

for topic in top_topics:
    # Filter reviews for the current topic
    filtered_reviews = df_filtered.filter(df_filtered["Topic"] == topic)["reviews.text"].to_list()

    # Concatenate reviews into a single string for summarization
    concatenated_reviews = " ".join(filtered_reviews)

    # Tokenize and ensure it's within the token limit (e.g., 1024 tokens)
    inputs = summarizer.tokenizer(concatenated_reviews, return_tensors="pt", truncation=True, max_length=1024, padding="max_length")

    # Summarize the concatenated reviews (using the summarizer)
    summary = summarizer.model.generate(inputs["input_ids"], max_length=50, min_length=10, num_beams=2, early_stopping=True)

    # Decode the summary
    summary_text = summarizer.tokenizer.decode(summary[0], skip_special_tokens=True)

    summaries[topic] = summary_text


# Print summaries
print(summaries)



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{0: 'AmazonBasics AA AAA batteries have done well by me appear to have a good shelf life. Will buy again.Seem to work as well as name brand batteries at a much better price These batteries are very long lasting the price is', 1: "I like the Amazon line of Basics. Nothing fancy, they just work. Can't go wrong with Amazon basics. Great bang for your buck. Greatest deal on the planet! Cheaper than my local box store. Great value.", 2: 'The Amazon Echo is a really fun thing to play with. We always try to find new things to say to it which would make it say something funny. The Echo has been the most helpful in our home for creating shopping lists.', 3: 'The modified Android operating system with Amazon features built-in is great. It has a 1.3 gigahertz processor, meaning for such a small device that decently enough. Internal memory is 8 gigs but you can expand to', 4: "They don't last as long as the brand name but are good enough considering they are much cheaper. They seem to work okay so

In [ ]:
# Step 4: Print the summaries
for topic, summary in summaries.items():
   print(f"Topic {topic}: {summary}")

Topic 0: AmazonBasics AA AAA batteries have done well by me appear to have a good shelf life. Will buy again.Seem to work as well as name brand batteries at a much better price These batteries are very long lasting the price is
Topic 1: I like the Amazon line of Basics. Nothing fancy, they just work. Can't go wrong with Amazon basics. Great bang for your buck. Greatest deal on the planet! Cheaper than my local box store. Great value.
Topic 2: The Amazon Echo is a really fun thing to play with. We always try to find new things to say to it which would make it say something funny. The Echo has been the most helpful in our home for creating shopping lists.
Topic 3: The modified Android operating system with Amazon features built-in is great. It has a 1.3 gigahertz processor, meaning for such a small device that decently enough. Internal memory is 8 gigs but you can expand to
Topic 4: They don't last as long as the brand name but are good enough considering they are much cheaper. They seem

#### using gpt-3.5-turbo

In [1]:
from google.colab import userdata
OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')

In [5]:
import polars as pl

In [7]:
df = pl.read_csv("reviews_with_Positive_topics.csv")

In [8]:
df_filtered = df.filter(pl.col("Topic") != -1)

In [9]:
topic_counts = df_filtered.group_by("Topic").agg(pl.col("Topic").count().alias("count"))

# Sort by the count in descending order
topic_counts = topic_counts.sort("count", descending=True)
print(topic_counts)

shape: (46, 2)
┌───────┬───────┐
│ Topic ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3774  │
│ 1     ┆ 1172  │
│ 2     ┆ 648   │
│ 3     ┆ 624   │
│ 4     ┆ 554   │
│ …     ┆ …     │
│ 41    ┆ 90    │
│ 42    ┆ 85    │
│ 43    ┆ 83    │
│ 44    ┆ 83    │
│ 45    ┆ 80    │
└───────┴───────┘


In [10]:
# Step 2: Select the top_n topics
top_n = 8
top_topics = topic_counts.head(top_n)["Topic"]

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI(
    # This is the default and can be omitted
    api_key= userdata.get('OPENAI_API_KEY'),
)

# Define function for generating summaries
def get_completion2(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # You can adjust temperature for creativity
        max_tokens=150  # Limit summary length
    )
    return response.choices[0].message.content

In [3]:
from google.colab import userdata

In [13]:
# Max token limit for gpt-3.5-turbo
MAX_TOKENS = 4096  # Adjust based on model's token limit

# Function to split text into chunks of manageable token length
def chunk_reviews(text, max_tokens=MAX_TOKENS):
    # Estimate the number of tokens (simple word-based approximation)
    tokens = text.split()
    num_chunks = (len(tokens) // max_tokens) + 1
    chunks = [tokens[i * max_tokens: (i + 1) * max_tokens] for i in range(num_chunks)]
    return [' '.join(chunk) for chunk in chunks]

# Loop through topics and generate summaries
summaries = {}

for topic in top_topics:
    # Efficient filtering of reviews based on the current topic using Polars
    filtered_reviews = df_filtered.filter(pl.col("Topic") == topic)["reviews.text"].to_list()

    # Concatenate reviews into a single string for summarization
    concatenated_reviews = " ".join(filtered_reviews)

    # Check if the concatenated reviews exceed token limits
    if len(concatenated_reviews.split()) > MAX_TOKENS:
        # Split reviews into chunks
        chunks = chunk_reviews(concatenated_reviews, max_tokens=MAX_TOKENS)

        chunk_summaries = []
        for i, chunk in enumerate(chunks):
            prompt = f"""
            Your task is to generate a short summary of a product review from an ecommerce site.

            Summarize the review below, delimited by triple backticks in at most 20 words.

            Review: ```{chunk}```
            """
            response = get_completion2(prompt)
            chunk_summaries.append(response)
            # print(f"Chunk {i+1} Summary: {response}\n")

        # Combine summaries from all chunks
        summaries[topic] = " ".join(chunk_summaries)
    else:
        # Summarize the entire review if it's under the token limit
        prompt = f"""
        Your task is to generate a short summary of a product review from an ecommerce site.

        Summarize the review below, delimited by triple backticks in at most 20 words.

        Review: ```{concatenated_reviews}```
        """
        summary_text = get_completion2(prompt)
        summaries[topic] = summary_text
        # print(f"Summary for topic {topic}: {summary_text}\n")

# Print the summaries
for topic, summary in summaries.items():
    print(f"Topic {topic} Summary: {summary}\n")

Topic 0 Summary: AmazonBasics batteries offer comparable performance to name brands at a lower price, with long-lasting power and convenient packaging. AmazonBasics batteries offer excellent performance and value, rivaling major brands at a fraction of the cost. Highly recommended. Amazon Basics batteries offer excellent value, comparable performance to top brands, and are ideal for frequent battery users. Amazon Basics batteries offer comparable performance to major brands at a lower price, with convenient delivery. Amazon Basics batteries offer great value, comparable performance to name brands, and are cost-effective for bulk purchases. AmazonBasics batteries are affordable, long-lasting, and comparable to name brands, making them a great value purchase. AmazonBasics AAA batteries offer good quality and value, comparable to name brands, with convenient packaging and affordable pricing. AmazonBasics batteries offer great value, lasting as long as major brands at a lower price. Highly